# Get WBC Policy Documents
CBA make donwloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

over come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

### Setup

In [1]:
import os
import re
import requests
from urllib.parse import urlparse, urljoin
import shutil

from loguru import logger

from IPython.display import HTML
from bs4 import BeautifulSoup

### Scraper Code

Notes
1. Only searchs for links in div#wrapper
2. Special code to deal with ASPX/DynamicContent
3. Limited to

In [55]:
def fetch_content(url:str, cookie:str):
    ''' Get the contents of a URL using a logged in cookie'''
    session = requests.Session()

    # Send the request with the cookies
    try:
      response = session.get(url, cookies={'Cookie': cookie})
    
      if not response:
        logger.warning(f"No response for {url}")
        return ""
      if response.status_code != 200:
        logger.warning(f"Response {response.status} for {url}")
        return ""
    
    except Exception as e:
      logger.error(f"Error {repr(e)} for {url}")
      return ""
    
    return response.text

def save_url(url:str, download_path:str, cookie:str) -> str:
  '''Save the url to the download path and return the contents'''
  html_content = fetch_content(url,cookie)

  full_path =  download_path + urlparse(url).path
  directory = os.path.dirname(full_path)
  
  if not os.path.exists(directory):
    os.makedirs(directory) 

  logger.info(f"Downloading {url}")
  with open(full_path, 'w') as f:
      f.write(html_content)

  return html_content

def aspx_link_to_html_link(aspx_url, cookie) -> list[str]:
  '''Extract the html links from the aspx file'''
  
  aspx_content = fetch_content(aspx_url, cookie)

  # Links in dynamic content function
  pattern = r'LoadDynamicContent\(([^,)]+),([^,)]+),([^,)]+)\)'
  matches = re.findall(pattern, aspx_content)
  if len(matches) >0 :
    # return  [urljoin(arg1,arg2) for arg1, arg2, _ in matches]
    return  [(arg1 + arg2).replace('"', '').replace(' ', '') for arg1, arg2, _ in matches]
  
  # or links in an input tag
  pattern = r'<input[^>]*value="([^"]*\.html)"[^>]*>'
  matches = re.findall(pattern, aspx_content, re.DOTALL)
  if matches:
    return matches
  
  logger.warning(f"{aspx_url} no html URL")
  return []
  
def fetch_html_links(html_page, class_names):
    '''Find all the links in the page'''
    soup = BeautifulSoup(html_page)

    links = []
    for class_name in class_names:
        for link in soup.select('div#wrapper a'):
                    if href := link.get('href'):
                        if href.startswith("/Net/Documentum/"):
                            href = domain+href
                        if not href.endswith('.aspx'):
                            print("Not an ASPX", href)
                        links.append(href) 
    return links

cache = set()
domain = r"https://www.commbroker.com.au"
def scrape(url:str, cookie, download_path:str, depth:int, starting_page:bool, search_only) -> None:
    # Stop if when we get to the bottom
    if depth and depth <= 0:
        logger.info(f"Bottomed out at {url}")
        return 
    
    path = urlparse(url).path
    extension = os.path.splitext(path)[-1].lower()
    
    # skip already processed urls   
    if path in cache:
        return
    
    cache.add(path)

    if not url.startswith(domain):
        logger.info(f"Skipping domain {url}")
        return

    # Skip unwanted videos etc
    if extension not in [".html", ".aspx", ".pdf"]:
        logger.info(f"Skipping unwanted extention  {url}")
        return
    
    if "Logon.aspx" in path:
        return

    # If aspx find the base html then extract the links and recurse without saving content
    if extension == ".aspx":
        html_urls = aspx_link_to_html_link(url, cookie)
        for html_url in html_urls:
            scrape(html_url, cookie, download_path, depth, starting_page, search_only)
        return
    
    # download all resources except starting page
    if starting_page:
        html_content = fetch_content(url,cookie)
    else:
        if search_only:
            logger.info(f"Saving {url}")           
            html_content = fetch_content(url,cookie)
        else:
            html_content = save_url(url, download_path, cookie)

    # if we have not yet exceeded the depth find all the links in wrapper and recurse
    if extension == ".html" and depth and depth > 0:
        soup = BeautifulSoup(html_content)
        for link in soup.select('div#wrapper a'):
            if href := link.get('href'):
                if not urlparse(href).netloc:
                    href = urljoin(domain, href)
                
                scrape(href, cookie, download_path, depth-1, starting_page=False, search_only=search_only)
    return

In [56]:
cookie="s_cc=true; s_nr=1733620003923; s_sq=%5B%5BB%5D%5D; bm_mi=8813A6ECDB9C1E8E53C3D981C88D970C~YAAQXgNiaDkfGdqTAQAAQrHv2xpfiPtvAV/a2BBMCbhd4TsxRf/4CJOb/Cwajy+tnLKRoQBYt1VAhzmUN0XRw/4LzUxo5qfRE6i5C2C3laMgemGT00sSSDycCXM9C4ZziMlQj7yuqzUSXVynrFduCLe1WyQm+CF5/MY7lmeI2uDGT8GRCxVN74QWtNHlWori8M8dVOPz7mlmaIcOthwj0xTW6CQxd/glBJAFg+ymWY2S2C1H9DSNGcXBRhK7OJqX32FAMAsvAumSlrDC7Sk3WF3/Ya5+CafBe3ytKxLrgF8rzd+U/unyWt7IzEKlVyxsZujJSKw+8t7AWfdK5A2Bs/ZH0d5hSDAyPrwyf5vByrsJ4Ium25x94J2u~1; ak_bmsc=CD06D2B46EEE1325CA25FCAFE2EC86AF~000000000000000000000000000000~YAAQRQNiaLXD3tiTAQAAf7bv2xosPcRGo6bXAKskl9It0Ue1hHaaEDXXUIHoWn0zNncUVjJ+/mwyiZfuygvz3P3ao2xb0oLAIr8HkQFNyrrEjTPzSRVxVDkntxK9LRluAoolcd3oA0mTBnryxHvRHV7VkyILedOBdEFdzsCDH99hHcvWzTj64a3YAk0PF0MiLc0z02g8ziBrIaB2SiwzNnv/ZENieyuQmDYP7PK5XtuhZocoZ2K1Dbtu0LLOKe4jkWTbFcyul4BghwaNucHz8CsL6/4mR6AHKoR8ntwcaf4rPVGaCEIVmLs0iDcPYBUUoXa7NnNcEtmL/RFyRCfUvFO766rnSPl16zYvk05P8VFOCoHO3H+xwtfyrj/3z1uSP8bc7AZaizXy47EJG8HQpd+hWQjow53+xaYOSPmVRhkqmPUzr73pMrI+KfJsyiemGRZ5lXVzRI9Brs8/cVbuQWy+YNVQ1XmUcoEQXbFO8BGEg8KJ+xcgJ6FtBYQdEn+AGKijja++mOepniDA3zAl6L0=; OpenIdConnect.nonce.uVhVKXzufhslVXraj2lrt4BEzqZyqz6cBmZnRo0h8ds%3D=aHA4eWx0bUY1N2FKdTBaanB3RXJWd0VjLW9pN0t5aHp1b0JvM1pTWkhzTHYyYWU3bU1IQmxvaXNEd3VzSENlc1NHcHJzc0xhckdVZGVCOXEyRDd1VWZ2VmhSX1ZsVEhnZFh5T25aY0JvVncybUZBSDdKbzFCa2dGRGdURHpSMmRwbEU4RkpoSGdGME45YXNMc1hlTTN3eGxYZmRyaFBMWUZ6akxqUjJCd1lVdG9samw2VVFTcG1ES3NzS1FSUU9mSmtsZHY0Mjh3UlR4a3I2ZGU5SDJzOXVlM2dz; ASP.NET_SessionId=41zrdlpb4d03kstu4kbcyi5k; cbid=n1ZO0tHFCTLiIbx_vAxWP4MgEaxtv9LM5sEHFspOYN25NveVWwpIVt0ezr3sbzPIljkna1uG-SS5D0F9BCsT7-RnsEteiWNGvmAwhjBhTgceTgVIyE-pNyvL3tS8Nqh6pG94PF8iKMfMoNiJoMeoc4kssRVV-4i2826yJ-ieoDV1WaLxKhraJfGEuf1w2JXYDnAha5CIkOR5w5U0OPv8AOi9rrJSxC7ZXX2EbSlDvdcQnzTMu-NQyrfINBlgUBvRkqvV9muM91-pmGgQCav_SAh0luZz-mCxR2j9gmOSk-nlTEXUKBm8QtXKKTCI3XH1PzrTQikfjCMd-xq_rKo0jSRxWUADXaDlGNpwU8pz5KjOLeAHtWA91fikxMaJ9a_nymulRPb-tftVU9YXtWTrVCYHsZFG4ZXVOuERJcGMEYc8iiZA6Vf2a0WVclPtebtPHfYMPg0WhyfTktumSfU8HIoLmx1ckt8NHD3ubjT7cNxhDf9hihQWV4GiSJWjTumu1vKCgQAPAy7MZxaPWek1kN5zsd82ijAG_A6tJ5cE3acdar1hGwhk5TzVxQrxYqiCtiKOM6yJQDfSlhP9hbePPh6ZVaf5t-tiwoTLnfOhvt4xnO747Uml-03ZcgMeeKMf2XolOBmgoiRjG36o3uG4-S8i_xpa5NYp1vZMxRweiJh5rd7XwkGEWBoBoloKpHQ0SQR5gXLzuIKOoGj4PjQVn06NQbk_fsr1CIMnSRrECYIyAKGIsIv2qqQ1npBZja6y5x-0jQG9w_Kr6OEiuOQz6pMajRN_Cf_j5xnjU5pGorgh4bVDdaLHYQXAjsmohPbde_D2eqaag7sG47zQgBH3AYQoRJWRNdvIGCcFSn6O-x7Ogct04Ucyt2bbFCWkNECl4ONjDElMJyI3auqriAlK4dwUyCFus9y3d6w_M8lsu2gFuug3h3wwOBzdJlqnUKBYI1aviCscnHbNIklTOyXDTS4Wzw0fjcpTYjB_4VwxkjCihPfyiJMTOPndtRaljTNy89es9yVK8aO3Cmn3KvGJkclj4qHuWRcPNBevWs_WR5lmQJSZRDX0xC8eUHjH8uHX7fQCsOYSa5aVbl7ChKAQoOsuZ_b5WWuGkG6iGgfjH3q_1abSDbSWbswXrrhVjegko6fItU02SPfvyfL8yPF9u36kffNnEIi_dovuTdwovImRCiJgfcbZdz1Jywp4ej_KZKUWsKriFB4N2afAU7qlO6Gi3OnQrYYiCTvefMvO5D56Bji1HVmx2RL5c2GAPfNoGoEwMp1A8B4KJrxZnSezgeUxz0QT_jwY8rlp8kQ8h5f_FIER5xxhGB_cL1AtVOCXg7JVWCJwBrndvAACUw6sfjf8_26kAT-44Gi9zJU5NT04x-SAa9vpeNxEWsVzWqI_2-ophHvrfpFvtdXnuyl2Gd5q8WVYross3aXW7cSfh37XyzKwqWRxvpZjWli41j54z6M3aMKzAHTSlDEAjHrqBZ3LJvgQ1dXOZnhUwH9Rpck5YM7FomtkFi38S3t3UsuXSD0Jko5WtU7iPqJyr4jLYZrUsvoLQmikkYDjSMITlfHQaFBGOIaojloQqNwqvfWT9tSc21_cAuW_rvDE06a0fXAc1wonKs8abczULmf23NAsViHPh_pLe1KiUH4RlD6uOJCL67gjn-BD04p57K8TKg3za6CBNKTqJHXbL8s1FGleushjygnQnxQIatkiy7jUar4Sg9CktQVwSyWKxTC0c01jT-FKFjo5Z34YFICXefyrWSycbvFmm9Q6-hYDe4A608ruGkbJXL5-hiRAKOAKbKZcPQ6qxJ-S1YVN-jl06aehdZJKAzTbIykMnneyXuqnG-1mlj0yfqc7HdZt2XLLNt0JaeBZAYieJSLujwqEV3EhPpXVXzKm-e0JqXfGjk82Mx1co2037yIHvuh3RkwPAD_lOfnkXMsPb9ghy8VKfPDFxZ7SznEtGE6mlR4zqOjt4IyA6wYfHuUPWgl8_DuvltPMPWXeDSLu4Z0qHqeQYHSScno2klYaXC3glnSz_OSn-CZhmXvMC9ikyIqVxwykzQqiG5ZDIpqUXmLeq6CekK4C5pg-dFplREGQTa1DhTjfLLX3I6VBxWZ1MoqX0OXZRemm0cXdDPdGT6VUgszvx0-5ZWEMKSS-ds-PEoD7VIQ4BME66brWZFtrY-DK3Vqgu2KlmM6bgiT0rHZUKTTwAzb8G05eX8yRo_eHAdWY-ICkf-joxZ9XN1lQXj9OYu3bhpvWQZFML-u9pb_uIpbdMXAueaHtahr4ys_PXmVRWiRkV3dRgLmueis6GChnyy_5gK5ia9caCzFjp2s9vfU6RAUXcWdP6ICVtUCl1mHHe3Qg6z1GGsViCfpCbU8ibYnZxNulQ6BD8cN7_NleLfhMJmHfMGjNslNPjaq2mI3TRPI6uBtSPva455m3Xy81mqHkCB1mifVkBS8L2Z4BD-WVfTkfSCKLJriQ4JXSrupbVTrHrWYZUf6YkB13uAcvaWfQfcueBPz4rsrHlGuEf1L4k4-on2E8_2c4t2B6rzYRXJO2hx080cSvFErEywjs50sq0NVjHtSnxQggC3T800zMwADViG2tCDiw9jSIlhs6maTUywwf447JEa_i94qNj8F9QnbVX2H4zIDlZL_bSiL7rP0mG-xtTBDDlJ79J9F1Xw7qno5z16VDW4ZYXK8w8sKHWT9CBRLQeRRffZ5udCWE3d7SBJo6I7pOAr_lcPhJa3NQhwIKiqIf6P-aNlMkw4jUwyGhSfW0bqPuyl04OX20JQEYVQtPz6IXmxA2Vt5_j27mf1UmAXvF2M36RGTv3McvAXFaCdyWjQ931oTP7xKKFq2n0NIaKILdeAz7DcLDN_ZLAPkCjKl616Nuprt1G-jlgDhAhVdwjOcgDGfFwepS177UvmhLcRyXv2VGqwuNHvyIx9wBCpkKVULUIo5Oauz6Yx79f0FfmKqjbCAKtkm4o65FEp1qTvqEcRS-msicGRloWhmYRUB32Uj1kh7oZNaiybPaoyKGb-h4j47uuZBLzbsmnMAGWwpP4NfMcYGz7TREH7IECZBJo1TO5dWCoeiJALsvGmRcara8MPxOGoEOLoj6nlp2rdAQB7WqCTM3079qmqHsyFPyhU4PJBtWlzWHbHM0LIOltG5P6EsHsfDKPuK-r8clP3rtVTax4U0bRTIkr_4n5T3qTxuGCY58GNnRTfvyMoui_UPq086LsKChJTNUGX2v9k2E-tWTHlQXPADlZCSLMMkjh2asdkAq664aICAm_zvjSn-h8PxO0525SYmy15CyCar6mbzU_ejEewd5z-E9OSstcX8Tn7prIZpCRdM7-sz_--wmPhDJwwCLEjPJctTbEUIvubhHo2Bbd8NR8GqMLbcyw7AIH7pAnnnluVrdnguL4PVEW0XLe2JR8LTwuVHV; htsm=exp=1734565800~acl=/*~hmac=5dd3ac61d0081c333396aa79ff6f0ee7611518fa43e4a90906c01b19feb5ff5f; bm_sz=2750E668606F5AF8653BFA0C9E2D83EC~YAAQ1ETHF3JGFpuTAQAA4TES3BqWfN6IHE26NDVppRnPDorTr49CncEnmhF15/2lCePD380VniqLNNQWtJ3ebBctX0S4NIsY8+r8aCtkndroS/Iky7YPOFxPgwPx+s+k8GqRmPbnzlF4m+PaxrIAsngdisj2bwJbv9PWlJCWg2/8ITRH7v7RSnCj+/tLNKVDtA7bx/0/5QKdRs5QmXBS7n1HGOjaWo+LLFDIh1CjbWglaEM6BOxNs+glmZCq/qMLVvNo18vFd+f+9Vb2K6dnt6eeoeLybKRJTPjZPsmVV/ZlEFreJfGsKP/0A1u50R3cE6X1PRYaFh28yPjggmRX6DjZUCH8tK6eb5CkKJnJ10eDRs5pZWahqkC6CFPMc4tP3zMhKql6T38EGFN+SeOP/5g9v9lRhdREG4HgMxRWgybGuu+yLpSgTxn13Ohj2SapmfUi/UiqjuJc+C9ZQCrUasjGJSEgCc7eshHBgSgaPLUrcAjXsG5B8fVlRMY2eveEckK/MA==~3487301~4407860; bm_sv=84A2C86790871EBBFF747C24FF5C8FDD~YAAQ3ETHF5EVN7STAQAArDMS3Bom0B3cNdbLImPfgv8djpSMPa8S13ky0UkWNl82jUd/XFzz9UMLf9WrO+NUDuoh8VAQi8Xj2ao1V7sltrg5VLSuTgAfixW3DKTqcD4r7e5fIGVpNleR2Ieb4cr+qHvFn8RXvSECHsq/Ah/xHsggs9ZIeVbLLzq+K3uTkE+6QGAyTec++Lt5sEzp7DeQ7c62307Rw4VMFiLjlpjM58J1tcIQAtmj1CMHtIqCW9xNPgt97iUmMpY=~1; _abck=5FD508811D6FE13655588BDF9BE601CC~-1~YAAQ1ETHF4ZGFpuTAQAA4DkS3A345SollweduU3FyDH/EzGbiM7kMRMcXK8vtlP9fM8OUrX4C2JDVGjb9JHdoPq2ULLbQJLrtbwTdgEZJg+X2cgrRZ71LqSocbsalYZGjFx61WEGc6udVGu8UZaFsCbfIvneHoT6FgqbUIn6T1t6WndU2RUSqPGFLR4ngNdEXJ3JjAwvUVPIUGyBk/7u+t9EpuAlJpHCB3tKXYX7c7ZG+iSSybeMOQiM2nzZIhfn3JG2MZcmYJuyASM9EmL07ujvXrl5tvIjsgQ4SekCjfbtO6x/f/9/f3TjV0fLwLYG8vcJ58vkiFMSDu8hzKmcCC9BIKWfUdlmkpkIq+evAfsuDlJfajqOyKc0TfymoWTa7ly/Ztu4z6i8Fb3Pg+MfnUDWt+S48J/ATZmPBoXEO9Dh0mgCrJLgF3CXn+VfqnLLkOk4987ELDx/e1KpjUGUMGPZoUf1JxgseXD1wSxCrOF5R/llsV+ua4opftb5QlwNlXeSkOlzuOi3+iEVNbQ=~-1~||0||~-1"
directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", 
       cookie, depth=3, download_path=directory_path, 
       starting_page=True, search_only=True)

2024-12-19 07:42:09.388 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default1.aspx no html URL
2024-12-19 07:42:09.666 | INFO     | __main__:scrape:114 - Saving https://www.commbroker.com.au/content/commbroker/en/private/latest-news/live-updates.html
2024-12-19 07:42:10.410 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default.aspx no html URL
2024-12-19 07:42:10.411 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/support-for-home-loan-customers.html?ei=pers_HL
2024-12-19 07:42:10.412 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/coronavirus.html
2024-12-19 07:42:10.412 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/home-loan-direct-debit-repayments.html
2024-12-19 07:42:10.413 | INFO     | __main__:scrape:114 - Saving https://www.commbroker.com.au/content/dam/commbroker/priva

# Download Comm Broker

In [58]:
directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"
cookie="s_cc=true; s_nr=1733620003923; s_sq=%5B%5BB%5D%5D; bm_mi=8813A6ECDB9C1E8E53C3D981C88D970C~YAAQXgNiaDkfGdqTAQAAQrHv2xpfiPtvAV/a2BBMCbhd4TsxRf/4CJOb/Cwajy+tnLKRoQBYt1VAhzmUN0XRw/4LzUxo5qfRE6i5C2C3laMgemGT00sSSDycCXM9C4ZziMlQj7yuqzUSXVynrFduCLe1WyQm+CF5/MY7lmeI2uDGT8GRCxVN74QWtNHlWori8M8dVOPz7mlmaIcOthwj0xTW6CQxd/glBJAFg+ymWY2S2C1H9DSNGcXBRhK7OJqX32FAMAsvAumSlrDC7Sk3WF3/Ya5+CafBe3ytKxLrgF8rzd+U/unyWt7IzEKlVyxsZujJSKw+8t7AWfdK5A2Bs/ZH0d5hSDAyPrwyf5vByrsJ4Ium25x94J2u~1; ak_bmsc=CD06D2B46EEE1325CA25FCAFE2EC86AF~000000000000000000000000000000~YAAQRQNiaLXD3tiTAQAAf7bv2xosPcRGo6bXAKskl9It0Ue1hHaaEDXXUIHoWn0zNncUVjJ+/mwyiZfuygvz3P3ao2xb0oLAIr8HkQFNyrrEjTPzSRVxVDkntxK9LRluAoolcd3oA0mTBnryxHvRHV7VkyILedOBdEFdzsCDH99hHcvWzTj64a3YAk0PF0MiLc0z02g8ziBrIaB2SiwzNnv/ZENieyuQmDYP7PK5XtuhZocoZ2K1Dbtu0LLOKe4jkWTbFcyul4BghwaNucHz8CsL6/4mR6AHKoR8ntwcaf4rPVGaCEIVmLs0iDcPYBUUoXa7NnNcEtmL/RFyRCfUvFO766rnSPl16zYvk05P8VFOCoHO3H+xwtfyrj/3z1uSP8bc7AZaizXy47EJG8HQpd+hWQjow53+xaYOSPmVRhkqmPUzr73pMrI+KfJsyiemGRZ5lXVzRI9Brs8/cVbuQWy+YNVQ1XmUcoEQXbFO8BGEg8KJ+xcgJ6FtBYQdEn+AGKijja++mOepniDA3zAl6L0=; OpenIdConnect.nonce.uVhVKXzufhslVXraj2lrt4BEzqZyqz6cBmZnRo0h8ds%3D=aHA4eWx0bUY1N2FKdTBaanB3RXJWd0VjLW9pN0t5aHp1b0JvM1pTWkhzTHYyYWU3bU1IQmxvaXNEd3VzSENlc1NHcHJzc0xhckdVZGVCOXEyRDd1VWZ2VmhSX1ZsVEhnZFh5T25aY0JvVncybUZBSDdKbzFCa2dGRGdURHpSMmRwbEU4RkpoSGdGME45YXNMc1hlTTN3eGxYZmRyaFBMWUZ6akxqUjJCd1lVdG9samw2VVFTcG1ES3NzS1FSUU9mSmtsZHY0Mjh3UlR4a3I2ZGU5SDJzOXVlM2dz; ASP.NET_SessionId=41zrdlpb4d03kstu4kbcyi5k; cbid=n1ZO0tHFCTLiIbx_vAxWP4MgEaxtv9LM5sEHFspOYN25NveVWwpIVt0ezr3sbzPIljkna1uG-SS5D0F9BCsT7-RnsEteiWNGvmAwhjBhTgceTgVIyE-pNyvL3tS8Nqh6pG94PF8iKMfMoNiJoMeoc4kssRVV-4i2826yJ-ieoDV1WaLxKhraJfGEuf1w2JXYDnAha5CIkOR5w5U0OPv8AOi9rrJSxC7ZXX2EbSlDvdcQnzTMu-NQyrfINBlgUBvRkqvV9muM91-pmGgQCav_SAh0luZz-mCxR2j9gmOSk-nlTEXUKBm8QtXKKTCI3XH1PzrTQikfjCMd-xq_rKo0jSRxWUADXaDlGNpwU8pz5KjOLeAHtWA91fikxMaJ9a_nymulRPb-tftVU9YXtWTrVCYHsZFG4ZXVOuERJcGMEYc8iiZA6Vf2a0WVclPtebtPHfYMPg0WhyfTktumSfU8HIoLmx1ckt8NHD3ubjT7cNxhDf9hihQWV4GiSJWjTumu1vKCgQAPAy7MZxaPWek1kN5zsd82ijAG_A6tJ5cE3acdar1hGwhk5TzVxQrxYqiCtiKOM6yJQDfSlhP9hbePPh6ZVaf5t-tiwoTLnfOhvt4xnO747Uml-03ZcgMeeKMf2XolOBmgoiRjG36o3uG4-S8i_xpa5NYp1vZMxRweiJh5rd7XwkGEWBoBoloKpHQ0SQR5gXLzuIKOoGj4PjQVn06NQbk_fsr1CIMnSRrECYIyAKGIsIv2qqQ1npBZja6y5x-0jQG9w_Kr6OEiuOQz6pMajRN_Cf_j5xnjU5pGorgh4bVDdaLHYQXAjsmohPbde_D2eqaag7sG47zQgBH3AYQoRJWRNdvIGCcFSn6O-x7Ogct04Ucyt2bbFCWkNECl4ONjDElMJyI3auqriAlK4dwUyCFus9y3d6w_M8lsu2gFuug3h3wwOBzdJlqnUKBYI1aviCscnHbNIklTOyXDTS4Wzw0fjcpTYjB_4VwxkjCihPfyiJMTOPndtRaljTNy89es9yVK8aO3Cmn3KvGJkclj4qHuWRcPNBevWs_WR5lmQJSZRDX0xC8eUHjH8uHX7fQCsOYSa5aVbl7ChKAQoOsuZ_b5WWuGkG6iGgfjH3q_1abSDbSWbswXrrhVjegko6fItU02SPfvyfL8yPF9u36kffNnEIi_dovuTdwovImRCiJgfcbZdz1Jywp4ej_KZKUWsKriFB4N2afAU7qlO6Gi3OnQrYYiCTvefMvO5D56Bji1HVmx2RL5c2GAPfNoGoEwMp1A8B4KJrxZnSezgeUxz0QT_jwY8rlp8kQ8h5f_FIER5xxhGB_cL1AtVOCXg7JVWCJwBrndvAACUw6sfjf8_26kAT-44Gi9zJU5NT04x-SAa9vpeNxEWsVzWqI_2-ophHvrfpFvtdXnuyl2Gd5q8WVYross3aXW7cSfh37XyzKwqWRxvpZjWli41j54z6M3aMKzAHTSlDEAjHrqBZ3LJvgQ1dXOZnhUwH9Rpck5YM7FomtkFi38S3t3UsuXSD0Jko5WtU7iPqJyr4jLYZrUsvoLQmikkYDjSMITlfHQaFBGOIaojloQqNwqvfWT9tSc21_cAuW_rvDE06a0fXAc1wonKs8abczULmf23NAsViHPh_pLe1KiUH4RlD6uOJCL67gjn-BD04p57K8TKg3za6CBNKTqJHXbL8s1FGleushjygnQnxQIatkiy7jUar4Sg9CktQVwSyWKxTC0c01jT-FKFjo5Z34YFICXefyrWSycbvFmm9Q6-hYDe4A608ruGkbJXL5-hiRAKOAKbKZcPQ6qxJ-S1YVN-jl06aehdZJKAzTbIykMnneyXuqnG-1mlj0yfqc7HdZt2XLLNt0JaeBZAYieJSLujwqEV3EhPpXVXzKm-e0JqXfGjk82Mx1co2037yIHvuh3RkwPAD_lOfnkXMsPb9ghy8VKfPDFxZ7SznEtGE6mlR4zqOjt4IyA6wYfHuUPWgl8_DuvltPMPWXeDSLu4Z0qHqeQYHSScno2klYaXC3glnSz_OSn-CZhmXvMC9ikyIqVxwykzQqiG5ZDIpqUXmLeq6CekK4C5pg-dFplREGQTa1DhTjfLLX3I6VBxWZ1MoqX0OXZRemm0cXdDPdGT6VUgszvx0-5ZWEMKSS-ds-PEoD7VIQ4BME66brWZFtrY-DK3Vqgu2KlmM6bgiT0rHZUKTTwAzb8G05eX8yRo_eHAdWY-ICkf-joxZ9XN1lQXj9OYu3bhpvWQZFML-u9pb_uIpbdMXAueaHtahr4ys_PXmVRWiRkV3dRgLmueis6GChnyy_5gK5ia9caCzFjp2s9vfU6RAUXcWdP6ICVtUCl1mHHe3Qg6z1GGsViCfpCbU8ibYnZxNulQ6BD8cN7_NleLfhMJmHfMGjNslNPjaq2mI3TRPI6uBtSPva455m3Xy81mqHkCB1mifVkBS8L2Z4BD-WVfTkfSCKLJriQ4JXSrupbVTrHrWYZUf6YkB13uAcvaWfQfcueBPz4rsrHlGuEf1L4k4-on2E8_2c4t2B6rzYRXJO2hx080cSvFErEywjs50sq0NVjHtSnxQggC3T800zMwADViG2tCDiw9jSIlhs6maTUywwf447JEa_i94qNj8F9QnbVX2H4zIDlZL_bSiL7rP0mG-xtTBDDlJ79J9F1Xw7qno5z16VDW4ZYXK8w8sKHWT9CBRLQeRRffZ5udCWE3d7SBJo6I7pOAr_lcPhJa3NQhwIKiqIf6P-aNlMkw4jUwyGhSfW0bqPuyl04OX20JQEYVQtPz6IXmxA2Vt5_j27mf1UmAXvF2M36RGTv3McvAXFaCdyWjQ931oTP7xKKFq2n0NIaKILdeAz7DcLDN_ZLAPkCjKl616Nuprt1G-jlgDhAhVdwjOcgDGfFwepS177UvmhLcRyXv2VGqwuNHvyIx9wBCpkKVULUIo5Oauz6Yx79f0FfmKqjbCAKtkm4o65FEp1qTvqEcRS-msicGRloWhmYRUB32Uj1kh7oZNaiybPaoyKGb-h4j47uuZBLzbsmnMAGWwpP4NfMcYGz7TREH7IECZBJo1TO5dWCoeiJALsvGmRcara8MPxOGoEOLoj6nlp2rdAQB7WqCTM3079qmqHsyFPyhU4PJBtWlzWHbHM0LIOltG5P6EsHsfDKPuK-r8clP3rtVTax4U0bRTIkr_4n5T3qTxuGCY58GNnRTfvyMoui_UPq086LsKChJTNUGX2v9k2E-tWTHlQXPADlZCSLMMkjh2asdkAq664aICAm_zvjSn-h8PxO0525SYmy15CyCar6mbzU_ejEewd5z-E9OSstcX8Tn7prIZpCRdM7-sz_--wmPhDJwwCLEjPJctTbEUIvubhHo2Bbd8NR8GqMLbcyw7AIH7pAnnnluVrdnguL4PVEW0XLe2JR8LTwuVHV; htsm=exp=1734565800~acl=/*~hmac=5dd3ac61d0081c333396aa79ff6f0ee7611518fa43e4a90906c01b19feb5ff5f; bm_sz=2750E668606F5AF8653BFA0C9E2D83EC~YAAQ1ETHF3JGFpuTAQAA4TES3BqWfN6IHE26NDVppRnPDorTr49CncEnmhF15/2lCePD380VniqLNNQWtJ3ebBctX0S4NIsY8+r8aCtkndroS/Iky7YPOFxPgwPx+s+k8GqRmPbnzlF4m+PaxrIAsngdisj2bwJbv9PWlJCWg2/8ITRH7v7RSnCj+/tLNKVDtA7bx/0/5QKdRs5QmXBS7n1HGOjaWo+LLFDIh1CjbWglaEM6BOxNs+glmZCq/qMLVvNo18vFd+f+9Vb2K6dnt6eeoeLybKRJTPjZPsmVV/ZlEFreJfGsKP/0A1u50R3cE6X1PRYaFh28yPjggmRX6DjZUCH8tK6eb5CkKJnJ10eDRs5pZWahqkC6CFPMc4tP3zMhKql6T38EGFN+SeOP/5g9v9lRhdREG4HgMxRWgybGuu+yLpSgTxn13Ohj2SapmfUi/UiqjuJc+C9ZQCrUasjGJSEgCc7eshHBgSgaPLUrcAjXsG5B8fVlRMY2eveEckK/MA==~3487301~4407860; bm_sv=84A2C86790871EBBFF747C24FF5C8FDD~YAAQ3ETHF5EVN7STAQAArDMS3Bom0B3cNdbLImPfgv8djpSMPa8S13ky0UkWNl82jUd/XFzz9UMLf9WrO+NUDuoh8VAQi8Xj2ao1V7sltrg5VLSuTgAfixW3DKTqcD4r7e5fIGVpNleR2Ieb4cr+qHvFn8RXvSECHsq/Ah/xHsggs9ZIeVbLLzq+K3uTkE+6QGAyTec++Lt5sEzp7DeQ7c62307Rw4VMFiLjlpjM58J1tcIQAtmj1CMHtIqCW9xNPgt97iUmMpY=~1; _abck=5FD508811D6FE13655588BDF9BE601CC~-1~YAAQ1ETHF4ZGFpuTAQAA4DkS3A345SollweduU3FyDH/EzGbiM7kMRMcXK8vtlP9fM8OUrX4C2JDVGjb9JHdoPq2ULLbQJLrtbwTdgEZJg+X2cgrRZ71LqSocbsalYZGjFx61WEGc6udVGu8UZaFsCbfIvneHoT6FgqbUIn6T1t6WndU2RUSqPGFLR4ngNdEXJ3JjAwvUVPIUGyBk/7u+t9EpuAlJpHCB3tKXYX7c7ZG+iSSybeMOQiM2nzZIhfn3JG2MZcmYJuyASM9EmL07ujvXrl5tvIjsgQ4SekCjfbtO6x/f/9/f3TjV0fLwLYG8vcJ58vkiFMSDu8hzKmcCC9BIKWfUdlmkpkIq+evAfsuDlJfajqOyKc0TfymoWTa7ly/Ztu4z6i8Fb3Pg+MfnUDWt+S48J/ATZmPBoXEO9Dh0mgCrJLgF3CXn+VfqnLLkOk4987ELDx/e1KpjUGUMGPZoUf1JxgseXD1wSxCrOF5R/llsV+ua4opftb5QlwNlXeSkOlzuOi3+iEVNbQ=~-1~||0||~-1"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cache=set()
scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx" ,    cookie, depth=3, download_path=directory_path, starting_page=True, search_only=False)
scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", cookie, depth=3, download_path=directory_path, starting_page=True, search_only=False)

2024-12-19 07:45:45.499 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default1.aspx no html URL
2024-12-19 07:45:45.920 | INFO     | __main__:scrape:114 - Saving https://www.commbroker.com.au/content/commbroker/en/private/latest-news/live-updates.html
2024-12-19 07:45:46.179 | INFO     | __main__:save_url:32 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/latest-news/live-updates.html
2024-12-19 07:45:46.967 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default.aspx no html URL
2024-12-19 07:45:46.968 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/support-for-home-loan-customers.html?ei=pers_HL
2024-12-19 07:45:46.969 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/coronavirus.html
2024-12-19 07:45:46.970 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/h